In [40]:
import dotenv
import os

dotenv.load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [41]:
import PyPDF3

book_path = '../data/raw/paes-historia-2023/economia_y_sociedad.pdf'

pdf = open(book_path, 'rb')
pdfdoc = PyPDF3.PdfFileReader(pdf)

In [42]:
from langchain_community.document_loaders import PyPDFLoader

loader:PyPDFLoader = PyPDFLoader(book_path)
pages = loader.load_and_split()

In [43]:
complete_text = ' '.join([page.page_content.replace('\n', ' ') for page in pages])
len(complete_text.split(' '))

752

In [44]:
from langchain.prompts import ChatPromptTemplate

prompt = """Eres un profesor experto en todas las materias del colegio.
Extrae las preguntas que encuentres en el siguiente texto:
texto = {texto}
y separalas con 2 punto y coma (;;)
En caso de que la pregunta venga con contexto, incluyelo con la pregunta.
Ejemplo: Pregunta 1;;Pregunta 2;;Pregunta 3
"""
prompt = ChatPromptTemplate.from_template(prompt)

In [45]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

In [46]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

llm_chain = (
    {"texto": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [47]:
output = llm_chain.invoke({"texto": complete_text})
output

'59. Una de las metas que tiene el Banco Central de Chile es promover la estabilidad y eficacia del sistema financiero. ¿Cuál es una acción que realiza esta entidad para cumplir dicho objetivo? A) Regular la cantidad de dinero circulante. B) Establecer los requisitos para solicitar un crédito. C) Fiscalizar los mercados de bancos, valores y seguros. D) Sancionar el incumplimiento en el pago de impuestos. E) Fijar los precios de los productos de consumo masivo.;;60. El funcionamiento del sistema económico chileno está determinado por una serie de factores; entre ellos, el consumo que hacen las familias como agentes de la economía. Considerando la realidad chilena actual, ¿cuál de las siguientes situaciones favorece este consumo? A) El mayor acceso a créditos. B) El aumento del ahorro previsional. C) La estabilidad de las tasas de interés. D) La existencia de una Ley del Consumidor. E) El establecimiento de una banda de precios.;;61. En una economía de mercado, cuando el precio de un bie

In [48]:
output.split(';;')

['59. Una de las metas que tiene el Banco Central de Chile es promover la estabilidad y eficacia del sistema financiero. ¿Cuál es una acción que realiza esta entidad para cumplir dicho objetivo? A) Regular la cantidad de dinero circulante. B) Establecer los requisitos para solicitar un crédito. C) Fiscalizar los mercados de bancos, valores y seguros. D) Sancionar el incumplimiento en el pago de impuestos. E) Fijar los precios de los productos de consumo masivo.',
 '60. El funcionamiento del sistema económico chileno está determinado por una serie de factores; entre ellos, el consumo que hacen las familias como agentes de la economía. Considerando la realidad chilena actual, ¿cuál de las siguientes situaciones favorece este consumo? A) El mayor acceso a créditos. B) El aumento del ahorro previsional. C) La estabilidad de las tasas de interés. D) La existencia de una Ley del Consumidor. E) El establecimiento de una banda de precios.',
 '61. En una economía de mercado, cuando el precio de